# Uncertainty in Software Architecture. Analisys of Apache Spark issue tracker.

In [2]:
from decisions import connect_to_mongo, get_decisions_filtered
db = connect_to_mongo()
decisions = get_decisions_filtered(db)
print("Total number of issues:%d"%(len(decisions)))

Total number of issues:3122


In [ ]:
decisions = [d for d in decisions if "description_annotations" in [key for key, value in d.items()]] 
print("Total number of issues annotated:%d"%len(decisions))

In [ ]:
for d in decisions:
    d["number_of_uncertainties"] = 0
    d["uncertainties"] = dict()
    for annotation in d.get("description_annotations"):
        if annotation["ner"]=="UNCERTAINTY":
            d["number_of_uncertainties"] += 1 
            try: 
                d["uncertainties"][annotation["token"]] += 1
            except KeyError as e:
                d["uncertainties"].update({annotation["token"] : 1})
    print(d["key"],d["number_of_uncertainties"],d["fields"]["issuetype"]["name"], d["uncertainties"])

In [ ]:
print("Max number of uncertainties in one issue", max([d["number_of_uncertainties"] for d in decisions]))

In [ ]:
uncers_by_issue_type = dict()
number_of_issues_with_uncer = dict()
total_uncers_by_issue_type = dict()
for d in decisions:
    try: 
        uncers_by_issue_type[d['fields']['issuetype']['name']] += d["number_of_uncertainties"]
        number_of_issues_with_uncer[d['fields']['issuetype']['name']] += 1 if d["number_of_uncertainties"] > 0 else 0
        total_uncers_by_issue_type[d['fields']['issuetype']['name']] += 1
    except KeyError as e:
        uncers_by_issue_type.update({d['fields']['issuetype']['name'] : d["number_of_uncertainties"]})
        number_of_issues_with_uncer.update({d['fields']['issuetype']['name'] : 1 if d["number_of_uncertainties"] > 0 else 0})
        total_uncers_by_issue_type.update({d['fields']['issuetype']['name'] : 1})        

# print("Total number of issues by issue type:")
# print(total_by_issue_type)

print("Number of issues that has at least one uncertainty:")
print(number_of_issues_with_uncer)

print("Number of uncertainties by issue type:")
print(uncers_by_issue_type)

# print("Ratio of uncertainty per issue type")
# for key,value in total_by_issue_type.items():
#     print("%s:%.2f"%(key,number_of_issues_with_uncer[key]/value*100))
    


In [ ]:
print("Total number of uncertainties")
print (sum([d["number_of_uncertainties"] for d in decisions]))

In [4]:
import discoverer

uncertainties = [
    "think", "thought", "thinking", "almost", 
    "apparent", "apparently", "appear", "appeared", "appears", "approximately", "around",
    "assume", "assumed", "certain amount", "certain extent", "certain level", "claim",
    "claimed", "doubt", "doubtful", "essentially", "estimate",
    "estimated", "feel", "felt", "frequently", "from our perspective", "generally", "guess",
    "in general", "in most cases", "in most instances", "in our view", "indicate", "indicated",
    "largely", "likely", "mainly", "may", "maybe", "might", "mostly", "often", "on the whole",
    "ought", "perhaps", "plausible", "plausibly", "possible", "possibly", "postulate",
    "postulated", "presumable", "probable", "probably", "relatively", "roughly", "seems",
    "should", "sometimes", "somewhat", "suggest", "suggested", "suppose", "suspect", "tend to",
    "tends to", "typical", "typically", "uncertain", "uncertainly", "unclear", "unclearly",
    "unlikely", "usually", "broadly", "tended to", "presumably", "suggests",
    "from this perspective", "from my perspective", "in my view", "in this view", "in our opinion",
    "in my opinion", "to my knowledge", "fairly", "quite", "rather", "argue", "argues", "argued",
    "claims", "feels", "indicates", "supposed", "supposes", "suspects", "postulates"
]
ents = ['UNCERTAINTY']
decision = [d for d in decisions if d['key']=='SPARK-22180'][0]

print("Decision title: %s \nDecision link: https://issues.apache.org/jira/browse/%s"%(decision['fields']['summary'], decision['key']))

print("Description:")
discoverer.main(decision['fields']['description'], *uncertainties, ents=ents)


Decision title: Allow IPv6 address in org.apache.spark.util.Utils.parseHostPort 
Decision link: https://issues.apache.org/jira/browse/SPARK-22180
Description:


In [ ]:
print("Comments:")
for key, val in enumerate(decision['comments']):
    print("Comment %d (Author: %s)"%(key, val['author']['name']))
    print(discoverer.main(val['body'], *uncertainties, ents=ents))

In [1]:
from openpyxl import Workbook
wb = Workbook()
ws1 = wb.create_sheet("test")
wb.save('balances.xlsx')
